# Preparing inputs and variables

## Import

In [ ]:
#imports
from osgeo import gdal
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
import arcpy
from arcpy import env
from arcpy.sa import *

## Folder paths

In [ ]:
# Folder path
folder_path = os.path.dirname(os.path.abspath("__file__"))

#import dir
iDirname = r"{0}\import".format(folder_path)
#export dir
eDirname = r"{0}\export".format(folder_path)

print(folder_path,iDirname,eDirname,sep='\n')

## Variables
- input files
- variables
- output files

In [ ]:
# inputs
boundary_shp = iDirname+"\\task1\\"+"La_palma_bounds.shp" #admin. limits
pop_raster = iDirname+"\\JRC_GRID_2018\\"+"JRC_1K_POP_2018.tif" # population raster data 2018
raster_tif = iDirname+"\\task1\\"+"clip_compbands_20210930.tif"
lavaflow_clip = iDirname+"\\clip\\"+"lava_flow.shp" # volcano lava region identified
#print(boundary_shp)

In [ ]:
# variables
boundary_fieldname = "NAME_4"

In [ ]:
# output
copy_boundary_shp=eDirname+"\\c_boundary.shp"
zonal_stats_pop_table=eDirname+"\\zonal_stats_pop.dbf"
zonal_stats_pop_shp=eDirname+"\\zs_pop.shp"
raster_mask=eDirname+"\\mask.tif"



# Set ArcGIS enviroment

In [ ]:
# SET ARCGIS ENVIRONMENT
arcpy.env.workspace = iDirname
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 UTM Zone 28N")
# Output fields are unqualified, so the field name will not contain the origin table
arcpy.env.qualifiedFieldNames = False
arcpy.env.scratchWorkspace = eDirname
#Allow for overwriting
arcpy.env.overwriteOutput = True

# Population data analysis

Process:
- create a copy of the original shp
- Calculate administrative boundary area in square km
- create Statistical Zone table by population data
- Join population to administrative limit boundary
- Calculate density by administrative zones (mean pop/ area)

Visualize data

## Process

### create feature copy

In [ ]:
# copy shapefile to export folder
arcpy.CopyFeatures_management(boundary_shp, copy_boundary_shp)
print("copy of the boundary completed")

### Calculate boundary area (square km)

In [ ]:
# add field area
arcpy.management.AddField(copy_boundary_shp, "area", "DOUBLE")
arcpy.management.AddField(copy_boundary_shp, "den_pop", "DOUBLE")
print("created field area")
field_names = [f.name for f in arcpy.ListFields(copy_boundary_shp)]
print(field_names)

# calculate area
expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")        
arcpy.CalculateField_management(copy_boundary_shp, "area", expression1, "PYTHON", )
print("calculated area finished")

### Statistical Zone

In [ ]:
# Execute ZonalStatisticsAsTable
ZonalStatisticsAsTable(copy_boundary_shp, boundary_fieldname, pop_raster, 
                                 zonal_stats_pop_table, "", "ALL", "CURRENT_SLICE")
print("statistical zone table finished")

### Join statistic data to boundary and export shp

In [ ]:
# Add join
fcjoin=arcpy.management.AddJoin(copy_boundary_shp, boundary_fieldname, zonal_stats_pop_table, boundary_fieldname)
field_names = [f.name for f in arcpy.ListFields(fcjoin)]
print(field_names)


In [ ]:
# export feature with join
arcpy.CopyFeatures_management(fcjoin, zonal_stats_pop_shp)
print("Join and export feature finished")

## Density calculation and Visualization

In [ ]:
# calculate density
arcpy.management.CalculateField(zonal_stats_pop_shp, 'den_pop', "$feature.MEAN / $feature.area", "ARCADE")

In [ ]:
# read shp on geopandas
shapefile = gpd.read_file(zonal_stats_pop_shp)

In [ ]:
# plot map
fig,ax = plt.subplots(figsize=(10,10))
shapefile.plot(ax=ax, column="den_pop", cmap="Blue")
ax.axis("off")


# Lava flow
- band composition and visualisation to identify lava flow
- Create mask: extract pixels value to quantify area of lava flow

In [ ]:
def normalize_MinMax(x):
    return((x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x)))

## create band composition

In [ ]:
ds= gdal.Open(raster_tif)
b4=ds.GetRasterBand(3).ReadAsArray()
#b8=ds.GetRasterBand(4).ReadAsArray()
b11=ds.GetRasterBand(5).ReadAsArray()

b4 = normalize_MinMax(b4)
#b8 = normalize_MinMax(b8)
b11 = normalize_MinMax(b11)
ds= None

## print map of raster band composition

In [ ]:
b_comp=np.dstack((b11,b11,b4))
#plt.imshow(b_comp)
#plt.show()

# zoom to region
plt.imshow(b_comp)
plt.xlim(700, 1700)
plt.ylim(3000,2400)
plt.show()

## Extract area of lava flow
- read band 11 from tiff
- clip the area of the lava using a shapefile to avoid getting cloud pixels
- extract the lava flow area by extracting pixel values grater then 7000 and export mask as tif

In [ ]:
# read b11
b11_tif=arcpy.management.MakeRasterLayer(raster_tif, "rdlayer", "", "", "5")
# clip image area
b11_tif_clip=ExtractByMask(b11_tif, lavaflow_clip)
# extract by atributed and save maska s tiff
attExtract=ExtractByAttributes(b11_tif_clip, "VALUE > 7000")
attExtract.save(raster_mask)

### Calculate lava flow area
- I was not able to calculate the area.
- This was done manually by opening the mask on GIS software and checking atribute table to get the number of pixels.

#### Area
Total number of pixels = 1123
pixel size = 10m
Lava flow area = 1123 x 100sqm = 112,300 sqm = 0.1123 sqkm

In [ ]:
# failed attempt to calculate area by counting number of pixels
raster_numpy_array = arcpy.RasterToNumPyArray(attExtract)
cnt_value = numpy.count_nonzero(raster_numpy_array,) 
print(cnt_value)